In [1]:
import json
import os
import random
import subprocess
import time
from datetime import datetime
from pathlib import Path

import numpy
from IPython.display import clear_output, display
from PIL import Image, ImageDraw
from tqdm import tqdm

In [2]:
def shell(cmd, debug=False, returns_POpen=False, close_fds=False):
    if debug:
        print(cmd)

    if returns_POpen:
        process = subprocess.Popen(cmd, close_fds=close_fds)
        return process

    process = subprocess.run(cmd, stdout=subprocess.PIPE)
    return process.stdout.decode("utf-8").strip().splitlines()

In [3]:
def adb(cmd, debug=False, returns_POpen=False, close_fds=False):
    # This function runs adb commands on your connected device or emulator.
    if type(cmd) == str:
        cmd = cmd.split(" ")
    cmd = ["adb"] + cmd
    return shell(cmd, debug=debug, returns_POpen=returns_POpen, close_fds=close_fds)


adb("wait-for-device")

[]

In [4]:
def pullPhoneScreen(resize_ratio=None, as_numpy=False, print_times=True):
    s = time.time()
    adb("shell screencap -p /sdcard/screen.png")
    adb("pull /sdcard/screen.png ./game.png")
    adb("shell rm /sdcard/screen.png")
    im = Image.open("game.png")
    im = im.convert("RGB")
    if resize_ratio is not None:
        im = im.resize(
            (int(im.width * resize_ratio), int(im.height * resize_ratio)),
            Image.Resampling.LANCZOS,
        )
    if print_times:
        print("pull image took ", time.time() - s)
    if as_numpy:
        return numpy.array(im)
    return im


def yieldPhoneScreen():
    while True:
        yield pullPhoneScreen()

In [5]:
def PillowToCv2(img):
    nimg = np.array(img)
    im = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    return im

In [18]:
import cv2
import numpy as np


class Beholder:
    def __init__(self, generator):
        self.generator = generator()
        self.matchers = []
        self.threshhold = 0.95

    def loadMatcher(self, name, filename):
        if Path(filename).exists():
            self.matchers.append((name, cv2.imread(filename, 0)))
        else:
            print("File is missing")

    def readNextImage(self):
        self.image = PillowToCv2(next(self.generator))
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

    def findMatches(self):
        self.readNextImage()
        matches = []
        for name, template in self.matchers:
            result = cv2.matchTemplate(self.gray, template, method=cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if max_val > self.threshhold:
                height, width = template.shape[:2]

                top_left = max_loc
                center = (top_left[0] + width / 2, top_left[1] + height / 2)
                matches.append((name, center))
        return matches

In [19]:
a = Beholder(yieldPhoneScreen)

In [22]:
a.loadMatcher("Button", "./templates/Button.png")
a.loadMatcher("UnfinishedSmithing", "./templates/forge_item.png")
a.loadMatcher("FinishedArrow", "./templates/FinishedArrow.png")

In [23]:
while True:
    matches = a.findMatches()
    print(matches)
    time.sleep(1)

pull image took  1.2242190837860107
[('FinishedArrow', (1993.0, 891.0), -0.565473198890686, 0.9999648928642273, (977, 67), (1938, 864))]
pull image took  1.0950653553009033


KeyboardInterrupt: 